Q1: Implementing an RNN for Text Generation

Task: Recurrent Neural Networks (RNNs) can generate sequences of text. You will train an LSTM-based RNN to predict the next character in a given text dataset.

1.	Load a text dataset (e.g., "Shakespeare Sonnets", "The Little Prince").
2.	Convert text into a sequence of characters (one-hot encoding or embeddings).
3.	Define an RNN model using LSTM layers to predict the next character.
4.	Train the model and generate new text by sampling characters one at a time.


In [ ]:
import tensorflow as tf
import numpy as np
import os

# Load Shakespeare dataset
path_to_file = tf.keras.utils.get_file('shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')
# Unique characters
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Vectorized text
text_as_int = np.array([char2idx[c] for c in text])
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Create batches of sequences
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

# Create input-target pairs
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True,
                             stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])

model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)
EPOCHS = 10

history = model.fit(dataset, epochs=EPOCHS)
def generate_text(model, start_string, temperature=1.0, num_generate=500):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    model.reset_states()

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = predictions[:, -1, :] / temperature  # temperature scaling
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
Length of text: 1115394 characters
Epoch 1/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 24s 66ms/step - loss: 2.8038
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 68ms/step - loss: 1.8215
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 67ms/step - loss: 1.5765
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 68ms/step - loss: 1.4597
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 70ms/step - loss: 1.3855
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 18s 72ms/step - loss: 1.3385
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 71ms/step - loss: 1.2914
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 70ms/step - loss: 1.2595
Epoch 9/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 70ms/step - loss: 1.2244
Epoch 10/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 70ms/step - loss: 1.1906


Q2: NLP Preprocessing Pipeline
Write a Python function that performs basic NLP preprocessing on a sentence. The function should do the following steps:
1.	Tokenize the sentence into individual words.
2.	Remove common English stopwords (e.g., "the", "in", "are").
3.	Apply stemming to reduce each word to its root form.
Use the sentence:
"NLP techniques are used in virtual assistants like Alexa and Siri."
The function should print:
•	A list of all tokens
•	The list after stop words are removed
•	The final list after stemming


In [ ]:
import spacy
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Load the English spaCy model
nlp = spacy.load("en_core_web_sm")

# First-time download of NLTK stopwords and stemmer (if not already downloaded)
import nltk
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('stemmers/porter')
except LookupError:
    nltk.download('porter')


def preprocess_nlp_spacy(sentence):
    # 1. Tokenization using spaCy
    doc = nlp(sentence)
    tokens = [token.text for token in doc]
    print("Tokens (spaCy):", tokens)

    # 2. Remove stopwords (using NLTK stopwords for consistency with original code)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    print("After Stopword Removal:", filtered_tokens)

    # 3. Stemming (using NLTK PorterStemmer for consistency)
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    print("After Stemming:", stemmed_tokens)

# Example sentence
sentence = "NLP techniques are used in virtual assistants like Alexa and Siri."
preprocess_nlp_spacy(sentence)

Tokens (spaCy): ['NLP', 'techniques', 'are', 'used', 'in', 'virtual', 'assistants', 'like', 'Alexa', 'and', 'Siri', '.']
After Stopword Removal: ['NLP', 'techniques', 'used', 'virtual', 'assistants', 'like', 'Alexa', 'Siri', '.']
After Stemming: ['nlp', 'techniqu', 'use', 'virtual', 'assist', 'like', 'alexa', 'siri', '.']


[nltk_data] Error loading porter: Package 'porter' not found in index


Q3: Named Entity Recognition with SpaCy

Task: Use the spaCy library to extract named entities from a sentence. For each entity, print:
•	The entity text (e.g., "Barack Obama")
•	The entity label (e.g., PERSON, DATE)
•	The start and end character positions in the string
Use the input sentence:
"Barack Obama served as the 44th President of the United States and won the Nobel Peace Prize in 2009."

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
sentence = "Barack Obama served as the 44th President of the United States and won the Nobel Peace Prize in 2009."
doc = nlp(sentence)

for ent in doc.ents:
    print(f"Text: {ent.text}, Label: {ent.label_}, Start: {ent.start_char}, End: {ent.end_char}")


Text: Barack Obama, Label: PERSON, Start: 0, End: 12
Text: 44th, Label: ORDINAL, Start: 27, End: 31
Text: the United States, Label: GPE, Start: 45, End: 62
Text: the Nobel Peace Prize, Label: WORK_OF_ART, Start: 71, End: 92
Text: 2009, Label: DATE, Start: 96, End: 100


Q4: Scaled Dot-Product Attention

Task: Implement the scaled dot-product attention mechanism. Given matrices Q (Query), K (Key), and V (Value), your function should:
•	Compute the dot product of Q and Kᵀ
•	Scale the result by dividing it by √d (where d is the key dimension)
•	Apply softmax to get attention weights
•	Multiply the weights by V to get the output
Use the following test inputs:
Q = np.array([[1, 0, 1, 0], [0, 1, 0, 1]])
K = np.array([[1, 0, 1, 0], [0, 1, 0, 1]])
V = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])


In [ ]:
import numpy as np
import math

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=-1, keepdims=True)

def scaled_dot_product_attention(Q, K, V):
    d = Q.shape[-1]
    scores = np.dot(Q, K.T) / math.sqrt(d)
    weights = softmax(scores)
    output = np.dot(weights, V)
    return weights, output

Q = np.array([[1, 0, 1, 0], [0, 1, 0, 1]])
K = np.array([[1, 0, 1, 0], [0, 1, 0, 1]])
V = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])

weights, output = scaled_dot_product_attention(Q, K, V)
print("Attention Weights:\n", weights)
print("Output Matrix:\n", output)


Attention Weights:
 [[0.73105858 0.26894142]
 [0.26894142 0.73105858]]
Output Matrix:
 [[2.07576569 3.07576569 4.07576569 5.07576569]
 [3.92423431 4.92423431 5.92423431 6.92423431]]


Q5: Sentiment Analysis using HuggingFace Transformers

Task: Use the HuggingFace transformers library to create a sentiment classifier. Your program should:
•	Load a pre-trained sentiment analysis pipeline
•	Analyze the following input sentence:
"Despite the high price, the performance of the new MacBook is outstanding."
•	Print:
o	Label (e.g., POSITIVE, NEGATIVE)
o	Confidence score (e.g., 0.9985)


In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
result = classifier("Despite the high price, the performance of the new MacBook is outstanding.")[0]

print(f"Sentiment: {result['label']}")
print(f"Confidence Score: {result['score']:.4f}")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Sentiment: POSITIVE
Confidence Score: 0.9998
